In [1]:
import os
from pyciemss.Ensemble.interfaces import (
    load_and_sample_petri_ensemble,
)

In [2]:
DEMO_PATH = "notebook/integration_demo/"
ASKENET_PATH_1 = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"
ASKENET_PATH_2 = "test/models/AMR_examples/SIDARTHE.amr.json"

ASKENET_PATHS = [ASKENET_PATH_1, ASKENET_PATH_2]

In [3]:
def solution_mapping1(model1_solution: dict) -> dict:
    return model1_solution


def solution_mapping2(model2_solution: dict) -> dict:
    mapped_solution = {}
    mapped_solution["Susceptible"] = (
        model2_solution["Susceptible"]
        + model2_solution["Recognized"]
        + model2_solution["Threatened"]
    )

    mapped_solution["Infected"] = (
        model2_solution["Infected"]
        + model2_solution["Ailing"]
        + model2_solution["Diagnosed"]
    )

    # Model 1 doesn't include dead people, and implicitly assumes that everyone who is infected will recover.
    mapped_solution["Recovered"] = (
        model2_solution["Healed"] + model2_solution["Extinct"]
    )

    return mapped_solution


solution_mappings = [solution_mapping1, solution_mapping2]

## load_and_sample_petri_model

In [5]:
weights = [0.5, 0.5]
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run sampling
ensemble_samples = load_and_sample_petri_ensemble(
    ASKENET_PATHS, weights, solution_mappings, num_samples, timepoints
)

# Save results
ensemble_samples.to_csv(os.path.join(DEMO_PATH, "results_petri_ensemble/sample_results.csv"), index=False)

## load_and_calibrate_and_sample_petri_model

In [7]:
# data_path = os.path.join(DEMO_PATH, "data.csv")
# num_samples = 100
# timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# # Run the calibration and sampling
# calibrated_samples = load_and_calibrate_and_sample_petri_model(
#     ASKENET_PATH,
#     data_path,
#     num_samples,
#     timepoints=timepoints,
#     verbose=True,
# )

# # Save results
# calibrated_samples.to_csv(
#     os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
# )

iteration 0: loss = 37.94811773300171
iteration 25: loss = 34.9170196056366
iteration 50: loss = 32.69080424308777
iteration 75: loss = 32.41730046272278
iteration 100: loss = 32.44331431388855
iteration 125: loss = 32.692076683044434
iteration 150: loss = 32.61838483810425
iteration 175: loss = 32.56607246398926
iteration 200: loss = 33.014737367630005
iteration 225: loss = 32.58268332481384
iteration 250: loss = 32.89216685295105
iteration 275: loss = 31.961626768112183
iteration 300: loss = 32.6565682888031
